<br>Nina Nguyen</br>
<br>CPSC 5610 - AI</br>
<br>February 23, 2019</br>
<br>Assignment 2 (Problem 2)</br>

#### Job Shop Scheduling ####

**Short summary**

1. The problem consists of N *jobs* 1..N  each job has some number of *tasks*
1. For each job, the tasks must be done in sequence
1. Each task requires a *resource*,  and two tasks that use the same resource cannot be scheduled at the same time
1. All tasks take three time units to complete
1. All tasks are ready for execution at time 0, and must complete at or before time 15
  1. Which means that if any job has more five tasks, it cannot be scheduled, no matter what

There are some hints at the bottom of the file.

Input to the problem comes in a variable like this:


In [ ]:
# Job 1
#   (o11, r1)   (o12, r2) (o13, r3)
# Job 2
#   (o21, r1)  (o22, r2)
# Job 3
#   (o31, r3) (o32, r1) (o33, r2)
# Job 4
#   (o41, r4) (o42, r2)
#
# The ri are the resources

example = [[("o11", "r1"), ("o12", "r2"), ("o13", "r3")],
           [("o21", "r1"), ("o22", "r2")],
           [("o31", "r3"), ("o32", "r1"), ("o33", "r2")],
           [("o41", "r4"), ("o42", "r2")]]


----------------------------------------------

In [ ]:
from constraint import *

In [ ]:
###  Your implementation of solveCSP here:  takes a problem description as input, returns a 
###  list of solutions as output.  The method problem.getSolutions() returns a list of solutions.
###  Each solution is a dictionary -- key is a task, value is the value it is assigned by the CSP

def solveCSP(problemList):
    
    jobCount = len(problemList) #4
    assignC = (len(problemList)+1)
    bucket = range(jobCount)
    problem = Problem()

    for job in bucket:
        taskOnly = problemList[job]
        tasks = [tuple[0] for tuple in taskOnly]
        problem.addVariables(tasks,range(assignC))
            

    addOrderConstraint(problemList,problem)
    addMutexConstraint(problemList,problem)
    return problem.getSolutions()

#One task must come after another task in a Job
def addOrderConstraint(problemList,problem):
    for jobno in range(0,len(problemList)):
        tasksAndResource = problemList[jobno]
        tasks = [tuple[0] for tuple in tasksAndResource]
        for i in range(0,len(tasks) - 1):
            order = (tasks[i], tasks[i+1])
            problem.addConstraint(lambda t1,t2: t1 < t2,order)

#Two tasks using the same Resource cannot be scheduled at the same time
def addMutexConstraint(problemList,problem):
    resourceToTasks = {}
    for jobno in range(0,len(problemList)):
        tasksAndResource = problemList[jobno]
        for task in range(0,len(tasksAndResource)):
            tasks = tasksAndResource[task][0]
            resource = tasksAndResource[task][1]
            if resource not in resourceToTasks:
                resourceToTasks[resource] = []
            resourceToTasks[resource].append(tasks)
            
    for key,value in resourceToTasks.items():
        resourceList = value
        problem.addConstraint(AllDifferentConstraint(),resourceList)
            
def subtasks(problemList,jobno):
    for job in range(0,len(problemList)):
        tasksAndResource = problemList[jobno]
        tasks = [tuple[0] for tuple in tasksAndResource]
    return tasks
    

def resourceFor(subtask,problemList):
    value = str()
    for jobno in range(0,len(problemList)):
        resource = problemList[jobno]
        for i in range(0,len(resource)):
            if resource[i][0] == subtask:
                value = str(resource[i][1])
    return value


In [ ]:
def printSolution(problemList):
    solutions = solveCSP(problemList)
    if len(solutions) == 0:
        print("No solutions for this problem")
    else:
        print("There are " + str(len(solutions)) + " solutions.  Here is one:\n")
        solution = solutions[0]
        for jobno in range(0, len(problemList)): #4
            print("Schedule for Job " + str(jobno+1))
            for subtask in sorted(subtasks(problemList, jobno)):
                print("   Subtask " + subtask + " starts at " + str(solution[subtask]*3) + \
                      " ends at " + str(2 + int(solution[subtask]*3)) + "\tuses " + resourceFor(subtask, problemList))
                

In [ ]:
printSolution(example)

-----------------------------------

**Hints**
1.  Since all tasks take three time units, it's easiest to partition time into "buckets" each of which is three time units long.  For example, bucket 1 is the interval between t=0 and t=2 inclusive, bucket 1 is t=3 to t=5 inclusive.  You will notice from the **printSolution** code that my solver does that bucketing, then when it prints the solution it converts the time bucket number to a real point in time
2.  There are only two kinds of constraints:  order constraints (one task must come after another) and mutex constraints (two tasks must not be scheduled at the same time).   If you write helper functions for those two constraints, it will simplify your code a lot.